<a href="https://colab.research.google.com/github/PTson2207/House-Prices-Tutorial/blob/main/House_Prices_Tutorial_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install catboost

     |████████████████████████████████| 65.8MB 55kB/s 


In [2]:
!pip install eli5

     |████████████████████████████████| 112kB 5.3MB/s 


In [3]:
!pip install hyperopt

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib
import seaborn as sns
sns.set()

from catboost import CatBoostRegressor, Pool, cv

from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.feature_selection import RFECV

import eli5
from eli5.sklearn import PermutationImportance

import hyperopt

from numpy.random import RandomState
from os import listdir

Using matplotlib backend: agg


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/test.csv')
sample_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/sample_submission.csv')

In [6]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [7]:
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [8]:
print(train.shape, test.shape)

(1460, 81) (1459, 80)


## Data Exploration

    Visualize data

In [9]:
plt.figure(figsize=(20,5))
sns.histplot(data=train, x=train["SalePrice"], color="tomato")
plt.title("Độ phân phối của SalePrice")
plt.ylabel("Tỉ trọng")

Text(0, 0.5, 'Tỉ trọng')

In [10]:
train[train['SalePrice'] <= 0]

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice


    Không có giá trị SalePrice <= 0

In [11]:
plt.figure(figsize=(20,5))
sns.distplot(np.log(train.SalePrice), color="tomato")
plt.title("Độ phân phối của các biến tập trung")
plt.ylabel("Tỉ trọng");

/usr/local/lib/python3.6/dist-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)


In [12]:
train["LogSalePrice"] = train['SalePrice'].apply(np.log)

    Drop nan-feature

In [13]:
combined = train.drop(["SalePrice", "LogSalePrice"],axis=1).append(test)
nan_percentage = combined.isnull().sum().sort_values(ascending=False)/combined.shape[0]
missing_val = nan_percentage[nan_percentage > 0]

In [14]:
plt.figure(figsize=(20,5))
sns.barplot(x=missing_val.index.values, y=missing_val.values*100, palette="Reds_r");
plt.title("% của các missing values");
plt.ylabel("%");
plt.xticks(rotation=90);

    Tìm những feature categorical nằm trong numerical

In [15]:
num_cadidates = list(combined.dtypes[combined.dtypes != "object"].index.values)
num_cadidates

['Id',
 'MSSubClass',
 'LotFrontage',
 'LotArea',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'TotRmsAbvGrd',
 'Fireplaces',
 'GarageYrBlt',
 'GarageCars',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal',
 'MoSold',
 'YrSold']

    Trong các feature số này có biến nào thực sự là đối tượng số thực sự hay không?
    Feature area là feature năng số nhưng MSSubClass thì sao?
    Nó chứa các nhóm khác nhau không phải là số mà là phân loại.

In [16]:
unique_counts = combined.loc[:, num_cadidates].nunique().sort_values()

plt.figure(figsize=(20, 5))
sns.barplot(unique_counts.index, unique_counts.values, palette="Oranges_r")
plt.xticks(rotation=90)
plt.yscale("log")

/usr/local/lib/python3.6/dist-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


In [17]:
cat_candidates = list(combined.dtypes[combined.dtypes == "object"].index.values)

In [18]:
num_to_cats = ["BsmtHalfBath", "HalfBath", "KitchenAbvGr", "BsmtFullBath", "Fireplaces", "FullBath", "GarageCars",
               "BedroomAbvGr", "OverallCond", "OverallQual", "TotRmsAbvGrd", "MSSubClass", "YrSold", "MoSold", 
               "GarageYrBlt", "YearRemodAdd"]

for feat in num_to_cats:
    num_cadidates.remove(feat)
    cat_candidates.append(feat)
    combined[feat] = combined[feat].astype("object")
    train[feat] = train[feat].astype("object")
    test[feat] = test[feat].astype("object")

In [19]:
num_cadidates

['Id',
 'LotFrontage',
 'LotArea',
 'YearBuilt',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'PoolArea',
 'MiscVal']

In [20]:
len(num_cadidates)

21

    các biến phân loại vô ích

        Trước tiên cần phải lọc ra cột category

In [21]:
cat_candidates

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition',
 'BsmtHalfBath',
 'HalfBath',
 'KitchenAbvGr',
 'BsmtFullBath',
 'Fireplaces',
 'FullBath',
 'GarageCars',
 'BedroomAbvGr',
 'OverallCond',
 'OverallQual',
 'TotRmsAbvGrd',
 'MSSubClass',
 'YrSold',
 'MoSold',
 'GarageYrBlt',
 'YearRemodAdd']

In [22]:
len(cat_candidates)

59

In [23]:
cat_candidates = combined.dtypes[combined.dtypes == "object"].index.values
len(cat_candidates)

59

In [24]:
frequencies = []
for col in cat_candidates:
    overall_freq = combined.loc[:, col].value_counts().max()/combined.shape[0]
    frequencies.append([col, overall_freq])

frequencies = np.array(frequencies)
freq_df = pd.DataFrame(index=frequencies[:,0], data=frequencies[:,1], columns=["frequency"])
sorted_freq = freq_df.frequency.sort_values(ascending=False)

plt.figure(figsize=(20,5))
sns.barplot(x=sorted_freq.index[0:30], y=sorted_freq[0:30].astype(np.float), palette="Blues_r")
plt.xticks(rotation=90)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 <a list of 30 Text major ticklabel objects>)

    Insights

In [25]:
example = "Utilities"
combined.loc[:, example].value_counts()

AllPub    2916
NoSeWa       1
Name: Utilities, dtype: int64

In [26]:
example = "Street"
combined.loc[:, example].value_counts()

Pave    2907
Grvl      12
Name: Street, dtype: int64

In [27]:
example = "Condition2"
combined.loc[:, example].value_counts()

Norm      2889
Feedr       13
Artery       5
PosN         4
PosA         4
RRNn         2
RRAn         1
RRAe         1
Name: Condition2, dtype: int64

In [28]:
cats_to_drop = ["Utilities"]

In [29]:
combined = combined.drop(cats_to_drop, axis=1)
train = train.drop(cats_to_drop, axis=1)
test = test.drop(cats_to_drop, axis=1)

In [30]:
cat_candidates = combined.dtypes[combined.dtypes=="object"].index.values
cat_candidates

array(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
       'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond',
       'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'YrSold',
       'SaleType', 'SaleCondition'], dtype=object)

In [31]:
len(cat_candidates)

58

    Hợp nhất các biến phân loại

In [32]:
def build_map(useless_levels, plugin_level, train_levels):
    plugin_map = {}
    for level in useless_levels:
        plugin_map[level] = plugin_level
    for level in train_levels:
        plugin_map[level] = level
    return plugin_map

In [33]:
def clean_test_levels(train, test):
    for col in test.columns:
        train_levels = set(train[col].unique())
        test_levels = set(test[col].unique())
        in_test_not_in_train = test_levels.difference(train_levels)
        if len(in_test_not_in_train)>0:
            close_to_mean_level = train.groupby(col).LogSalePrice.mean() - train.SalePrice.apply(np.log).mean()
            close_to_mean_level = close_to_mean_level.apply(np.abs)
            plugin_level = close_to_mean_level.sort_values().index.values[0]
            in_test_not_in_train = list(in_test_not_in_train)
            plugin_map = build_map(in_test_not_in_train, plugin_level, train_levels)
            test[col] = test[col].map(plugin_map)
    
    return train, test

In [34]:
test["MSSubClass"].value_counts()

20     543
60     276
50     143
120     95
30      70
70      68
160     65
80      60
90      57
190     31
85      28
180      7
75       7
45       6
40       2
150      1
Name: MSSubClass, dtype: int64

# Data Preparation

    Dealing with outliers

In [35]:
fig, ax = plt.subplots(len(num_cadidates),3,figsize=(20,len(num_cadidates)*6))

for n in range(len(num_cadidates)):
    feat = num_cadidates[n]
    ax[n,0].scatter(train[feat].values, np.log(train.SalePrice.values), s=4)
    ax[n,0].set_ylabel("Log SalePrice")
    ax[n,0].set_xlabel(feat);
    ax[n,1].scatter(np.log(train[feat].values+1), np.log(train.SalePrice.values), s=4)
    ax[n,1].set_ylabel("Log SalePrice")
    ax[n,1].set_xlabel("Log" + feat);
    sns.histplot(test[feat].dropna(), kde=True, ax=ax[n,2], color="limegreen")
    ax[n,2].set_title("Distribution in test")

    Dọn dẹp outlier

In [36]:
outlier_ids = set()
outlier_ids = outlier_ids.union(set(train[train["LotArea"]>60000].index.values))
outlier_ids = outlier_ids.union(set(train[train.LotFrontage > 200].index.values))
outlier_ids = outlier_ids.union(set(train[(train.LotFrontage > 150) & (train.SalePrice.apply(np.log) < 11)].index.values))
outlier_ids = outlier_ids.union(set(train[train.GrLivArea > 4500].index.values))
outlier_ids = outlier_ids.union(set(train[train["1stFlrSF"] > 4000].index.values))
outlier_ids = outlier_ids.union(set(train[train.MasVnrArea > 1400].index.values))
outlier_ids = outlier_ids.union(set(train[train["BsmtFinSF1"] > 5000].index.values))
outlier_ids = outlier_ids.union(set(train[train.TotalBsmtSF > 6000].index.values))
outlier_ids = outlier_ids.union(set(train[(train.OpenPorchSF > 500) & (np.log(train.SalePrice) < 11)].index.values))

In [37]:
outlier_ids

{249, 297, 313, 335, 451, 495, 523, 706, 934, 1298, 1337}

In [38]:
train.shape

(1460, 81)

In [39]:
train = train.drop(list(outlier_ids))
combined = combined.drop(list(outlier_ids))

In [40]:
train.shape

(1449, 81)

In [41]:
combined.shape

(2897, 79)

    Imputing missing values

In [42]:
def imputr_na_trees(df, col):
    if df[col].dtype == "object":
        df[col] = df[col].fillna("None")
        df[col] = df[col].astype("object")
    else:
        df[col] = df[col].fillna(0)
    return df

In [43]:
for col in combined.columns:
    combined = imputr_na_trees(combined, col)

In [44]:
num_cadidates = combined.dtypes[combined.dtypes != "object"].index.values

In [45]:
len(num_cadidates)
num_cadidates

array(['Id', 'LotFrontage', 'LotArea', 'YearBuilt', 'MasVnrArea',
       'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF',
       '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'GarageArea',
       'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',
       'ScreenPorch', 'PoolArea', 'MiscVal'], dtype=object)

In [46]:
cat_candidates = combined.dtypes[combined.dtypes == "object"].index.values
len(cat_candidates)

58

In [47]:
cat_candidates

array(['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape',
       'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle',
       'OverallQual', 'OverallCond', 'YearRemodAdd', 'RoofStyle',
       'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', 'BsmtFullBath',
       'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr',
       'KitchenAbvGr', 'KitchenQual', 'TotRmsAbvGrd', 'Functional',
       'Fireplaces', 'FireplaceQu', 'GarageType', 'GarageYrBlt',
       'GarageFinish', 'GarageCars', 'GarageQual', 'GarageCond',
       'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'MoSold', 'YrSold',
       'SaleType', 'SaleCondition'], dtype=object)

In [48]:
combined.isnull().sum().sum()

0

    --> tổng null = 0

    Make new columns


In [49]:
combined["TotalSF"] = combined['1stFlrSF'] + combined['2ndFlrSF'] + combined['TotalBsmtSF']
combined['GreenArea'] = combined["LotArea"] - combined['GrLivArea'] - combined["GarageArea"]

# CastBoot

    read paper:
        https://arxiv.org/pdf/1706.09516.pdf

    Hyper Method

In [50]:
def run_catboost(train, test, hold, params, n_splits=10, 
                  n_repeats=1, plot=False, use_features=None, plot_importance=True):
    folds = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_satate=1)
    p_hold = np.zeros(hold.shape[0])
    y_hold = hold.target
    p_test = np.zeros(test.shape[0])

    if use_features is None:
        use_features = test.columns.values
    
    cat_features = np.where(test.loc[:, use_features].dtypes=="object")[0]
    x_hold = hold.loc[:, use_features]
    x_test = test.loc[:, use_features]

    feature_importance_df = pd.DataFrame(index=use_features)

    m = 0
    cv_scores = []
    for train_idx, dev_idx in folds.splits(train):
        x_train, x_dev = train.iloc[train_idx][use_features], train.iloc[dev_idx][use_features]
        y_train, y_dev = train.target.iloc[train_idx], train.target.iloc[dev_idx]
        
        train_pool = Pool(x_train, y_train, cat_features=cat_features)
        dev_pool = Pool(x_dev, y_dev, cat_features=cat_features)
        model = CatBoostRegressor(**params)
        model.fit(train_pool, eval_set=dev_pool)

        p_hold += model.predict(x_hold)/(n_splits*n_repeats)
        log_p_test = model.predict(x_test)
        p_test += (np.exp(log_p_test)-1)/(n_splits*n_repeats)

        y_pred = model.predict(x_dev)
        feature_importance_df.loc[:, "fold_" + str(m)] = model.get_feature_importance(train_pool)
        cv_scores.append(np.sqrt(mse(y_dev, y_pred)))
        m+=1

    print("hold out rmse: " + str(np.sqrt(mse(y_hold, p_hold))))
    print("cv mean rmse: " + str(np.mean(cv_scores)))
    print("cv std rmse: " + str(np.std(cv_scores)))
    
    feature_importance_df["mean"] = feature_importance_df.mean(axis=1)
    feature_importance_df["std"] = feature_importance_df.std(axis=1)
    feature_importance_df = feature_importance_df.sort_values(by="mean", ascending=False)
    
    if plot_importance:
        plt.figure(figsize=(15,20))
        sns.barplot(x=feature_importance_df["mean"].values, y=feature_importance_df.index.values);
        plt.title("Feature importances");
        plt.show()
    
    results = {"last_model": model,
               "last_train_pool": train_pool,
               "feature_importance": feature_importance_df, 
               "p_hold": p_hold,
               "p_test": p_test,
               "cv_scores": cv_scores}
    return results

    Setup data

In [51]:
traindf = combined.iloc[0:train.shape[0]].copy()
traindf.loc[:, "target"] = train['LogSalePrice']
testdf = combined.iloc[train.shape[0]::].copy()

In [52]:
traindf, holddf = train_test_split(traindf, test_size=0.25, random_state=0)
print(traindf.shape, holddf.shape, testdf.shape)

(1086, 82) (363, 82) (1448, 81)


    parameter

In [53]:
org_params = {
    'iterations': 10000,
    'learning_rate': 0.08,
    'eval_metric': 'RMSE',
    'random_seed': 42,
    'logging_level': 'Silent',
    'use_best_model': True,
    'loss_function': 'RMSE',
    'od_type': 'Iter',
    'od_wait': 1000,
    'one_hot_max_size': 20,
    '12_leaf_reg': 100,
    'depth': 3,
    'rsm': 0.6,
    'random_strength': 2,
    'bagging_temperature': 10
    }